# 自定义工具及其应用
https://python.langchain.com/docs/use_cases/tool_use/prompting/

In [1]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
model_ty = Tongyi(temperature=0)
model_qf = QianfanLLMEndpoint(temperature=0)

In [3]:
import langchain
langchain.__version__

'0.1.9'

In [5]:
#! pip install --upgrade langchain -i https://mirrors.aliyun.com/pypi/simple

In [ ]:
#! pip install langchainhub -i https://mirrors.aliyun.com/pypi/simple

## 单个自定义工具及其使用

* 装饰器

Python装饰器起以下主要作用：

1. **添加功能**：装饰器能够在不修改原函数或类代码的基础上，为其添加新的功能。这包括但不限于日志记录、性能监控、权限控制、缓存、事务处理、数据验证、输入/输出格式化等。

2. **模块化与复用**：装饰器将通用的、与业务逻辑分离的功能封装为独立的模块（即装饰器函数），使得这些功能可以方便地应用于多个不同的函数或方法，提高代码的模块化程度和复用性。

3. **保持代码整洁**：使用装饰器可以避免在函数主体中混杂非核心逻辑，保持函数专注于其核心职责，从而提高代码的可读性和可维护性。通过在函数定义前使用`@装饰器名`的语法糖，装饰器的使用显得简洁且直观。

4. **无侵入式扩展**：装饰器允许在不直接修改已有代码的前提下，对函数的行为进行扩展。这意味着即使在后期需要添加、修改或移除装饰器提供的功能，也不会影响到函数本身的定义，有利于遵循“开闭原则”（Open/Closed Principle），即对扩展开放，对修改关闭。

5. **保留原函数接口**：装饰器返回的新函数通常会包裹原函数的调用，确保新函数对外暴露的接口（如参数列表、返回值类型）与原函数保持一致，用户无需因添加装饰器而调整对目标函数的调用方式。

6. **类型检查、日志记录、性能测试**：装饰器常用于进行类型检查以确保函数接收正确的参数类型；在函数执行前后插入日志语句，记录函数调用的相关信息，如函数名、参数、执行时间等，用于调试和性能分析；计算函数的执行时间，帮助识别和优化性能瓶颈。

7. **权限控制**：在Web开发中，装饰器可用于实现用户认证和授权，确保只有具备相应权限的用户才能访问特定的路由或资源。

综上所述，Python装饰器是一种强大的工具，用于在不破坏原有代码结构的情况下，为函数或类添加额外行为、执行预处理或后处理任务、统一管理横切关注点（cross-cutting concerns），从而提升代码的组织性、灵活性和可维护性。


### langchain中的tool

在LangChain中,`tool`装饰器的作用是将一个Python函数转换为`Tool`对象,以便在代理(Agent)执行时作为工具使用。

`Tool`对象描述了一个可执行的具体操作,包括名称、描述、函数指针以及函数所需的输入示例等信息。代理在执行时,可以根据当前的目标和环境,选择合适的工具并调用其函数来完成任务。

使用`tool`装饰器的步骤如下:

1. 定义一个Python函数,该函数完成某个特定的操作,如进行数学计算、查询API等。

2. 使用`@tool`装饰器对该函数进行装饰,提供一些关于该工具的元数据,如名称、描述、输入示例等。

3. 将装饰后的函数传递给各种langchain内置代理然后通过`AgentExecutor`执行。

例如:

```python
from langchain.tools import tool

@tool
def calculator(operation, num1, num2):
    """
    Performs a simple calculation with the given operation and numbers.
    """
    operations = {"加": lambda a, b: a + b,
                  "减": lambda a, b: a - b, 
                  "乘": lambda a, b: a * b,
                  "除": lambda a, b: a / b}
    
    try:
        op_func = operations[operation]
        result = op_func(num1, num2)
        return f"{num1} {operation} {num2} = {result}"
    except KeyError:
        return f"Operation {operation} not supported"
```

在上面的例子中,我们定义了一个`calculator`函数,用于执行基本的数学运算。通过`@tool`装饰器,该函数被转换为一个`Tool`对象,具有名称"calculator"、描述"Performs a simple calculation..."以及输入示例。然后,我们可以将该工具传递给代理:在执行时,代理会根据当前目标选择合适的工具,并调用其函数来完成任务。

总之,`tool`装饰器让我们可以方便地将Python函数转换为可供代理使用的工具,从而扩展代理的功能,使其能够执行更多种类的任务。

In [4]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """将两个整数相乘。"""
    return first_int * second_int

In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - 将两个整数相乘。
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [6]:
multiply.invoke({"first_int": 4, "second_int": 5})
#ChatTongyi()

20

In [7]:
from langchain.tools.render import render_text_description
rendered_tools = render_text_description([multiply])
rendered_tools

'multiply: multiply(first_int: int, second_int: int) -> int - 将两个整数相乘。'

In [8]:
from langchain_core.prompts import ChatPromptTemplate

rendered_tools = render_text_description([multiply])

system_prompt = f"""您是一名助理，可以使用以下工具集。 以下是每个工具的名称和说明:

{rendered_tools}

根据用户输入，返回要使用的工具的名称和输入。 将您的响应作为带有“name”和“arguments”键的 JSON blob 返回."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [9]:
bool(1)

True

### JsonOutputParser与自定义工具的参数传递

`JsonOutputParser`是Langchain中用于解析JSON输出的一个函数。它的作用是将AI生成的JSON格式的输出解析为Python对象,以便在后续处理中使用。

对于输入的JSON格式,`JsonOutputParser`要求输入为有效的JSON字符串。一般来说,该JSON字符串应该包含一个字典作为顶层对象,其中包含相关的键值对。

* `JsonOutputParser`是一个runable的函数，通常都用在链里负责给工具传递大模型回答所产生的参数



In [10]:
#例如,假设我们有一个AI模型生成了如下JSON输出:
json_str="""```json
{
  "result": "2 + 2 = 4",
  "mode": "math_operation"
}
```"""

In [11]:
from langchain_core.output_parsers import JsonOutputParser

#我们可以使用`JsonOutputParser`将其解析为Python字典:
#如您所见,`JsonOutputParser`将JSON字符串解析为了一个Python字典对象。

parser = JsonOutputParser()
output_dict = parser.parse('{"result": "2 + 2 = 4", "mode": "math_operation"}')
#output_dict = parser.parse(json_str)
print(output_dict)

{'result': '2 + 2 = 4', 'mode': 'math_operation'}


In [12]:
model = ChatTongyi()
chain = prompt | model 
#| JsonOutputParser()
chain.invoke({"input": "3乘以4等于"})

AIMessage(content='```json\n{\n  "name": "multiply",\n  "arguments": {\n    "first_int": 3,\n    "second_int": 4\n  }\n}\n```')

In [13]:
chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input": "3乘以4等于"})

12

### classwork 1

* 注册聚合数据账号账号并申请天气预报api接口，尝试在python中调用接口，相关指导视频如下：

https://www.juhe.cn/

* 基于此接口封装一个自定义工具

* 基于此工具完成一个出行天气顾问的代理应用

In [14]:
import requests
import json

In [29]:
#resp_json

In [15]:
@tool
def tianqi(city: str) -> dict:
    """查询最近几日的天气情况，包括温度，天气，湿度，风向等"""
    headers ={"Content-Type": "application/x-www-form-urlencoded"}
    url = "http://apis.juhe.cn/simpleWeather/query"
    params = {
        "key":"", # 在个人中心->我的数据,接口名称上方查看
        "city":city, # 要查询的城市名称或城市ID
    }
    resp = requests.get(url,params,headers=headers)
    resp_json = json.loads(resp.text)
    return resp_json

In [31]:
#tianqi.invoke({"city":"杭州"})

In [81]:
print(tianqi.name)
print(tianqi.description)
print(tianqi.args)

tianqi
tianqi(city: str) -> dict - 查询最近几日的天气情况，包括温度，天气，湿度，风向等
{'city': {'title': 'City', 'type': 'string'}}


In [64]:
#此JSON blob需要有如下格式：```json...```

In [22]:
template_tianqi ="""根据下面问题和json格式的响应，编写一个针对此问题自然语言回应：
问题：{question}
响应：{response}"""
prompt_response_tianqi = ChatPromptTemplate.from_template(template_tianqi)

In [17]:
#chain0.invoke({"input": "明天杭州出去需要带伞吗"})

In [24]:
chain1.invoke("女朋友明天要去北京玩，能否以我的口吻给她些出行建议，务必让她感动！")

'亲爱的，明天你去北京玩真是太棒了！记得带上一件轻薄的外套，因为白天温度大约在21度，但早晚可能会有些凉意。白天阳光明媚，空气质量相对较好，AQI为57，记得戴上口罩哦。未来几天天气预报显示都是晴朗的好天气，16号是西南风转北风，17号则是南风转东北风，记得适时调整衣物以防风。18号和19号气温会稍微升高，19号会有少许多云，但别担心，20号又会回归晴朗，温度适中。总的来说，北京的春天既美丽又宜人，记得拍下美美的照片分享给我，安全出行，玩得开心哦！'

### classwork 2

* 注册聚合数据账号账号并申请万年历api接口，尝试在python中调用接口，相关指导视频如下：

https://www.juhe.cn/

* 基于此接口封装一个自定义工具

* 基于此工具完成一个农历休假顾问的代理应用

In [25]:
@tool
def wannianli(date: str) -> dict:
    "根据指定日期查询其农历，习俗，星期几，假期，生肖"
    date=date.replace("-0","-")
    print(date)
    headers ={"Content-Type": "application/x-www-form-urlencoded"}
    url = "http://v.juhe.cn/calendar/day"
    params = {
        "key":"", # 在个人中心->我的数据,接口名称上方查看
        "date":date, # 指定日期,格式为YYYY-MM-DD,如月份和日期小于10,则取个位,如:2012-1-1
    }
    resp = requests.get(url,params,headers=headers)
    resp_json = json.loads(resp.text)
    return resp_json

In [39]:
wannianli.invoke({"date":'2024-04-08'})

2024-4-8


{'reason': 'Success',
 'result': {'data': {'animalsYear': '龙',
   'weekday': '星期一',
   'lunarYear': '甲辰年',
   'lunar': '二月三十',
   'year-month': '2024-4',
   'date': '2024-4-8',
   'suit': '结婚.出行.搬家.签订合同.交易.搬新房.开业.栽种.安床.挂匾.拆卸.出火.收养子女.开光',
   'avoid': '祈福.安葬.祭祀.作灶.入殓.探病',
   'holiday': '',
   'desc': ''}},
 'error_code': 0}

In [26]:
import time
# 获取当前日期
current_date = time.strftime("%Y-%m-%d", time.localtime())
print(current_date)

2024-04-16


In [27]:
current_date = time.strftime("%Y-%m-%d", time.localtime())
rendered_tools = render_text_description([wannianli])
system_prompt = f"""您是一名助理，可以使用以下工具集。 以下是每个工具的名称和说明:

{rendered_tools}

今天是{current_date},根据用户输入，返回要使用的工具的名称和输入。 将您的响应作为带有'name'和'arguments'键的 JSON blob 返回，“arguments”键对应的值应该是所选函数的输入参数的字典，字典里不要有任何说明,此JSON blob必须是如下格式：```json
...
```"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [28]:
chain0 = {"input":RunnablePassthrough()}|prompt | model | JsonOutputParser() | itemgetter("arguments") | wannianli
chain1 = {"question":RunnablePassthrough(), "response":chain0}|prompt_response_tianqi|model|StrOutputParser()

In [29]:
chain1.invoke("明天农历几号")

2024-4-17


'明天是农历2024年的甲辰年三月初九，星期三。适宜进行房屋清洁，但其他事情最好避免，因为有“诸事不宜”的说法。请注意日期是公历2024年4月17日。'

## 多个工具的使用

In [30]:
@tool
def add(first_int: int, second_int: int) -> int:
    "将两个整数相加。"
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "对底数求指数幂。"
    return base**exponent

In [31]:
tools = [add, exponentiate, multiply]


def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [32]:
current_date = time.strftime("%Y-%m-%d", time.localtime())
rendered_tools = render_text_description(tools)
system_prompt = f"""您是一名助理，可以使用以下工具集。 以下是每个工具的名称和说明:

{rendered_tools}

今天是{current_date},根据用户输入，返回要使用的工具的名称和输入。 将您的响应作为带有'name'和'arguments'键的 JSON blob 返回，“arguments”键对应的值应该是所选函数的输入参数的字典，字典里不要有任何说明,此JSON blob必须是如下格式：```json
...
```"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [33]:
chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input": "3加上1234"})

1237

### classwork 3

* 仿照上面多工具例子完成，在工具列表中加入

In [37]:
chain1.invoke("南京天气怎样")

'南京现在的天气是阴天，气温为23度，湿度为65%，风向是东风，风力等级为3级。未来几天的预报显示，16日和17日还是以阴天为主，温度在16到25摄氏度之间；18日和19日会有多云转阴和小雨转中雨的情况，温度分别在13到22度和17到22度；而20日则是阴天，风向会由东风转为东南风。请注意天气变化，适时增减衣物。'

In [62]:
print(rendered_tools)

add: add(first_int: int, second_int: int) -> int - 将两个整数相加。
exponentiate: exponentiate(base: int, exponent: int) -> int - 对底数求指数幂。
multiply: multiply(first_int: int, second_int: int) -> int - 将两个整数相乘。
tianqi: tianqi(city: str) -> dict - 查询最近几日的天气情况，包括温度，天气，湿度，风向等
wannianli: wannianli(date: str) -> dict - 根据指定日期查询其农历，习俗，星期几，假期，生肖


In [67]:
#tools

## 多工具并行使用

参考改编自：https://python.langchain.com/docs/use_cases/tool_use/parallel/

### classwork4 
* 参考上面链接代码完成工具并行运行的改编

In [38]:
from operator import itemgetter
from typing import Union

from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)



In [39]:
current_date = time.strftime("%Y-%m-%d", time.localtime())
rendered_tools = render_text_description(tools)
system_prompt = f"""您是一名助理，可以使用以下工具集。 以下是每个工具的名称和说明:

{rendered_tools}

今天是{current_date},根据用户输入，返回解决用户问题所需的所有工具的名称和输入，按照执行顺序。 将您的响应作为多个带有'name'和'arguments'键的 JSON blob 返回，“arguments”键对应的值应该是所选函数的输入参数的字典，字典里不要有任何说明,此JSON blob必须是如下格式：```json
...
```"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [41]:
chain0.invoke("明天是农历几号，杭州天气怎样，方便出行吗？")

2024-4-17


[{'name': 'wannianli',
  'arguments': {'date': '2024-04-17'},
  'output': {'reason': 'Success',
   'result': {'data': {'animalsYear': '龙',
     'weekday': '星期三',
     'lunarYear': '甲辰年',
     'lunar': '三月初九',
     'year-month': '2024-4',
     'date': '2024-4-17',
     'suit': '房屋清洁.馀事勿取.塞穴',
     'avoid': '诸事不宜',
     'holiday': '',
     'desc': ''}},
   'error_code': 0}},
 {'name': 'tianqi',
  'arguments': {'city': '杭州'},
  'output': {'reason': '查询成功!',
   'result': {'city': '杭州',
    'realtime': {'temperature': '24',
     'humidity': '73',
     'info': '阴',
     'wid': '02',
     'direct': '东风',
     'power': '2级',
     'aqi': '91'},
    'future': [{'date': '2024-04-16',
      'temperature': '15/24℃',
      'weather': '小雨转大到暴雨',
      'wid': {'day': '07', 'night': '23'},
      'direct': '东风转东北风'},
     {'date': '2024-04-17',
      'temperature': '13/18℃',
      'weather': '小雨',
      'wid': {'day': '07', 'night': '07'},
      'direct': '东北风'},
     {'date': '2024-04-18',
      'tempe

## 大模型的函数调用-Function call

* 借助了大语言模型对于代码的理解

https://blog.csdn.net/weixin_43679037/article/details/136198044

https://blog.csdn.net/jsjsjs1789/article/details/136949541


openai的工具案例：
https://python.langchain.com/docs/use_cases/tool_use/quickstart/

* 国内大模型通常没有bind_tools方法：model_with_tools = model.bind_tools([multiply], tool_choice="multiply")

* 工具调用还不支持

In [49]:
chat_qf = QianfanChatEndpoint(model="ERNIE-Bot")
chat_xh = ChatSparkLLM()

In [50]:
from langchain_core.utils.function_calling import convert_to_openai_tool

formatted_tools = [convert_to_openai_tool(tool) for tool in tools]

In [51]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [52]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(chat_qf, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [54]:
#请仔细与链接上的案例输出比较，这里是失败的它并没有使用工具！
agent_executor.invoke({"input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"})

## 